In [26]:
import pandas as pd

In [27]:
df = pd.read_csv(r"..\..\datasets\2. Depurados\TLC Aggregated Data\ML_TS_Output.csv")

In [28]:
df.head()

,ds,yhat,yhat_lower,yhat_upper,industry,column
0,2021-01-01,1.239464e+07,1.194575e+07,1.285786e+07,FHV - High Volume,total_trips
1,2021-02-01,1.255440e+07,1.209617e+07,1.302421e+07,FHV - High Volume,total_trips
2,2021-03-01,1.433721e+07,1.389973e+07,1.483875e+07,FHV - High Volume,total_trips
3,2021-04-01,1.383410e+07,1.336851e+07,1.432150e+07,FHV - High Volume,total_trips
4,2021-05-01,1.451887e+07,1.404423e+07,1.498482e+07,FHV - High Volume,total_trips


In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3956 entries, 0 to 3955
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   ds          3956 non-null   object 
 1   yhat        3956 non-null   float64
 2   yhat_lower  3956 non-null   float64
 3   yhat_upper  3956 non-null   float64
 4   industry    3956 non-null   object 
 5   column      3956 non-null   object 
dtypes: float64(3), object(3)
memory usage: 185.6+ KB


In [30]:
"""
Transforma el DataFrame de formato largo a formato amplio.
"""
# Convertir la columna 'ds' a datetime
df['ds'] = pd.to_datetime(df['ds'].str.split(' ').str[0])

# Realizar el pivot
df_pivot = df.pivot_table(index=['ds', 'industry'], columns='column', values=['yhat'], aggfunc='first')#, 'yhat_lower', 'yhat_upper'

# Aplanar las columnas multi-nivel generadas por el pivot
df_pivot.columns = [f'{col[1]}' for col in df_pivot.columns]

# Resetear el índice para obtener las columnas 'ds' y 'industry'
df_pivot.reset_index(inplace=True)

df_pivot['year'] = df_pivot['ds'].dt.year
df_pivot['days_in_month'] = df['ds'].dt.days_in_month


In [31]:
df_pivot.head()

,ds,industry,avg_trip_distance,total_amount,total_co2_emission,total_trips,unique_vehicles,year,days_in_month
0,2021-01-01,FHV - High Volume,3.575869,2.044370e+08,16792.527590,1.239464e+07,47019.331820,2021,31
1,2021-01-01,FHV - Other,3.435398,NaN,1763.465074,1.202691e+06,10214.675716,2021,28
2,2021-01-01,Green Taxi,3.522061,NaN,110.462784,8.548868e+04,991.248863,2021,31
3,2021-01-01,Total Mercado,3.263928,2.111750e+08,19618.440506,1.507310e+07,62427.784523,2021,30
4,2021-01-01,Yellow Taxi,2.411213,2.034171e+07,1258.382129,1.753982e+06,4624.998999,2021,31


In [32]:
# Variables necesarias pra Modelo de Flujo de Fondos
df_pivot['avg_income_per_vehicle'] = df_pivot['total_amount'] / df_pivot['unique_vehicles'] 
df_pivot['avg_vehicle_distance_per_month'] = df_pivot['total_trips'] *  df_pivot['avg_trip_distance'] / df_pivot['unique_vehicles'] 
#df_pivot['avg_hours_per_driver_per_month'] = df_pivot['avg_hours_per_day_per_driver'] * df_pivot['days_in_month']

In [33]:
# Agrupar por 'industry', 'column' y 'year' y calcular la suma de 'yhat', 'yhat_lower', 'yhat_upper'
df_agg = df_pivot.groupby(['industry', 'year'])[['avg_income_per_vehicle', 'avg_vehicle_distance_per_month', 'total_co2_emission']].sum().reset_index() #'avg_hours_per_driver_per_month'

In [34]:
df_agg.rename(
    columns={
        "industry": "industry",
        "year": "year",
        "avg_income_per_vehicle": "anual_income_per_vehicle",
        "avg_vehicle_distance_per_month": "anual_distance_per_vehicle",
        #"avg_hours_per_driver_per_month": "anual_hours_per_driver",
        "total_co2_emission": "anual_total_co2_emission",
    },
    inplace=True,
)

In [35]:
"""
Guarda el DataFrame de agregados anuales en un archivo CSV.
"""
df_agg.to_csv(r"..\..\datasets\2. Depurados\TLC Aggregated Data\ML_TS_Output_Anualized.csv", index=False)
